<a href="https://colab.research.google.com/github/OlgaTihanovich/DataScience/blob/main/HW5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# AdaBoost
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Генерируем искусственные данные
X, y = make_classification(n_samples=500, n_features=5, n_informative=2, n_redundant=2, random_state=42)
# n_samples - Количество образцов (наблюдений)
# n_features - Общее количество признаков (и информативных, и неинформативных, и дублирующихся)
# n_informative - Количество информативных признаков
# n_redundant - Количество избыточных признаков, которые содержат схожую с информативными информацию

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Создание и обучение классификатора AdaBoost
base_estimator = DecisionTreeClassifier(max_depth=2)
ada_classifier = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=100,
    learning_rate=0.15,
    random_state=42
)
ada_classifier.fit(X_train, y_train)

# Прогнозирование классов на тестовом наборе данных
y_pred = ada_classifier.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.95
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.94      0.95        53
           1       0.94      0.96      0.95        47

    accuracy                           0.95       100
   macro avg       0.95      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100

Confusion Matrix:
[[50  3]
 [ 2 45]]


In [3]:
# ExtraTreesClassifier
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

# Генерируем искусственные данные
X, y = make_classification(n_samples=500, n_features=5, n_informative=2, n_redundant=2, random_state=42)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Создание и обучение классификатора Extra Trees с использованием GridSearchCV для подбора гиперпараметров
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Инициализация ExtraTreesClassifier
clf = ExtraTreesClassifier(random_state=42)

# Настройка GridSearchCV
grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Обучение модели с подбором гиперпараметров
grid_search.fit(X_train, y_train)

# Вывод лучших параметров и точности
print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая точность (CV):", grid_search.best_score_)

# Использование лучшей модели для прогнозирования
best_clf = grid_search.best_estimator_
y_pred = best_clf.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy на тестовых данных: {accuracy:.2f}')
print('Classification Report:')
print(report)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Точность на обучающей выборке для проверки переобучения
y_train_pred = best_clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f'Training Accuracy: {train_accuracy:.2f}')

# Кросс-валидация для дополнительной оценки модели
cv_scores = cross_val_score(best_clf, X, y, cv=5, scoring='accuracy')
print(f'Кросс-валидационные оценки: {cv_scores}')
print(f'Средняя кросс-валидационная точность: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}')

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Лучшие параметры: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 100}
Лучшая точность (CV): 0.9574999999999999
Accuracy на тестовых данных: 0.95
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.94      0.95        50
           1       0.94      0.96      0.95        50

    accuracy                           0.95       100
   macro avg       0.95      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100

Confusion Matrix:
[[47  3]
 [ 2 48]]
Training Accuracy: 0.99
Кросс-валидационные оценки: [0.96 0.96 0.95 0.95 0.98]
Средняя кросс-валидационная точность: 0.96 ± 0.01
